This notebook uses iFeatureOmega, a feature generation software, to add to our feature space for a RandomForestClassifier that predicts protein pair functionality.

To do:

1) Write unit tests for iFeatureOmega
2) time trial for different descriptors

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.ensemble
import sklearn.feature_selection
import unittest
import iFeatureOmegaCLI
import Bio.SeqIO
import Bio.SeqRecord
import io
from io import StringIO
import time

In [2]:
cd /Users/loganroberts/Learn2Therm/ValidProt/FAFSA

/Users/loganroberts/Learn2Therm/ValidProt/FAFSA


In [3]:
#convert to pandas df
df = pd.read_csv('learn2therm_sample_50k.csv')

In [4]:
cd /Users/loganroberts/Learn2Therm/ValidProt/notebooks

/Users/loganroberts/Learn2Therm/ValidProt/notebooks


In [5]:
def get_fasta_from_dataframe(dataframe, output_file:str):
    #adjust this to write function with BioPython

    with open(output_file, 'w') as f:
        for _, row in df.iterrows():
            f.write('>{}\n{}\n'.format((row['meso_index']), row['m_protein_seq']))
    return output_file

In [6]:
get_fasta_from_dataframe(df, 'meso_50k.fasta')
df

,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,...,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len
0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,875,...,1153.0,27.5,50.0,22.5,MAESGTSRRADHLVPVPGPDAEPPAVADELLRAVGRGDEQAFGRLY...,MPSQITESERIELAERFERDALPLLDQLYSAALRMTRNPADAEDLV...,ECF RNA polymerase sigma factor SigK,sigma-70 family RNA polymerase sigma factor,206,202
1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,11324,...,1014.0,25.0,54.0,29.0,MARIALVDDDRNILTSVSMTLEAEGFEVETYNDGQSALDAFNKRMP...,MRVLLVEDDPNTSRSIEMMLTHANLNVYATDMGEEGIDLAKLYDYD...,response regulator transcription factor,response regulator transcription factor,233,237
2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,875,...,1138.0,28.0,50.0,22.0,MKDTVVFVTGAARGIGAHTARLAVARGARVALVGLEPHLLADLAAE...,MTPEQIFSGQTAIVTGGASGIGAATVEHIARRGGRVFSVDLSYDSP...,SDR family oxidoreductase,SDR family oxidoreductase,287,252
3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,875,...,1077.0,28.0,50.0,22.0,MTSGLWERVLDGVWVTIQLLVLSALLATAVSFVVGIARTHRLWIVR...,MAMSRRKRGQLARGIQYAILVIVVVVLALLADWGKIGKAFFDWEAA...,ectoine/hydroxyectoine ABC transporter permeas...,amino acid ABC transporter permease,234,269
4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,9827,...,991.0,30.0,50.0,20.0,MIISLRRGLRFIRFIVFFAALVYLFYHVLDLFNGWISPVDQYQMPT...,MKRMVWRTLKVFIIFIACTLLFYFGLRFMHLEYEQFHRYEPPEGPA...,YqzK family protein,YqzK family protein,80,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0.344828,0.169972,0.192308,180,0.509915,200,0.738007,53,14963,...,1055.0,28.0,52.5,24.5,MDNATFRLGDDLSVRLPGHSRWIGQVEREQRWLPWLAPRLPLTVST...,MPPQPRPLRPNDPREIGGFALLGRLGEGGQGTVYLGGAPDGRRVAV...,aminoglycoside phosphotransferase family protein,serine/threonine protein kinase,271,353
49996,49996,0.417989,0.389163,0.392060,190,0.935961,187,0.935000,314,7134,...,1083.0,26.5,52.5,26.0,MFRTGVKAEIGRSLAVVGEAEDVERAVRVVLEQRPDVVLLDVHLPG...,MILEAEPDIVVVGEAGDGEKAVEEARALQPDVVLMDIRMPRKDGVE...,response regulator transcription factor,response regulator transcription factor,200,203
49997,49997,0.352174,0.239645,0.242152,230,0.680473,227,0.685801,279,7134,...,991.0,28.0,52.5,24.5,MIRLAELTKTYPGQQHPAVDGISMEVAEGEIVVLVGPSGCGKTTTL...,MTEQPILSARGLTVDFRLRGGRRARAVDGVDLDLAPGEVLALAGES...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,331,338
49998,49998,0.334764,0.331915,0.329810,232,0.987234,230,0.966387,281,11324,...,907.0,29.0,54.0,25.0,MSESHAGALLSVRGLTAGYGGATALDGVSLTVAAGETVALLGANGA...,MSLLTTSGLTRHFSGIHAVEGVDFTLEAGEIRALIGSNGAGKTTLV...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,238,235


In [7]:
#getting an error in the try/except block


from Bio import Seq, SeqIO

def get_fasta_from_dataframe_biopython(dataframe, output_file:str):
    # Create a list of SeqRecord objects
    records = []
    for _, row in dataframe.iterrows():
        try:
            seq = Seq(row['m_protein_seq'])
        except Exception as e:
            print(f"Skipping row {row.name}: {str(e)}")
            continue
        record = SeqIO.SeqRecord(
            seq=seq,
            id=row['meso_index'],
            description=""
        )
        records.append(record)
    
    # Write the records to a FASTA file
    with open(output_file, 'w') as f:
        SeqIO.write(records, f, 'fasta')
    
    return output_file


In [8]:
def get_protein_descriptors(fasta:str, descriptors=[]):
    
    """
    Generates features from a protein sequence

    Parameters
    ----------
    Fasta file with protein sequences.

    Returns
    -------
    Vector of descriptors
    """
    
    #create iProtein object
    protein = iFeatureOmegaCLI.iProtein(fasta)
    
    #not sure why we need this yet. Right now it is stored in local directory.
    params = protein.import_parameters('protein_parameters.json')
    
    protein_descriptors = {}
    
    for descriptor in descriptors:
        protein.get_descriptor(descriptor)
        protein_descriptors.update({f'{descriptor}':protein.encodings})
        
    return protein_descriptors

In [9]:
#this function is only necessary if we get a fasta file with a descriptor generated from component 3

def remove_fasta_description(filename:str):
    
    """
    Removes description from fasta file so that iProtein can read the input.
    Might not be necessary in our current format.
    
    Parameters
    ----------
    Fasta file with protein sequences.

    Returns
    -------
    Fasta file with protein sequences, sans description.
    """
    
    #assign unwanted string to object
    string_to_remove = "<unknown description>"

    #open file
    with open(filename, "r") as file:
        content = file.read()
    
    # Remove the string
    new_content = content.replace(string_to_remove, "")

    #overwrite file without string
    with open(filename, "w") as file:
        seq = file.write(new_content)
        
    return seq

In [10]:
#combine both of the above functions

def fasta_to_descriptors(fasta:str, descriptors=[]):

    #remove description from fasta file
    remove_fasta_description(fasta)
    
    #return protein descriptors
    return get_protein_descriptors(fasta, descriptors=descriptors)

In [11]:
def create_new_dataframe(dataframe, output_file, descriptors=[]):
    """
    Creates new dataframe with descriptors added.
    
    Parameters
    ----------
    Pandas dataframe, list of descriptors as strings, output file name.

    Returns
    -------
    Dataframe including vector(s) of descriptors
    """
    
    fasta = get_fasta_from_dataframe(dataframe, output_file)
    
    feature_dict = get_protein_descriptors(fasta, descriptors)
    
    df = dataframe.reset_index()
    
    for desc in descriptors:
        
        feature_dict[desc].index = feature_dict[desc].index.astype(int)
        features = feature_dict[desc].reset_index()
        
        df = pd.merge(df, features, how='outer', left_index=True, right_index=True)
        
    return df

In [12]:
df = create_new_dataframe(df, 'meso_50k.fasta', descriptors=['AAC', 'GAAC'])

File imported successfully.


In [18]:
df['norm_bit_score_m'] = df['bit_score']/df['m_protein_len']
df['norm_bit_score_t'] = df['bit_score']/df['t_protein_len']

In [19]:
df

,index_x,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,...,AAC_W,AAC_Y,index,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge,norm_bit_score_m,norm_bit_score_t
0,0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,...,0.009709,0.029126,12897,0.451456,0.053398,0.165049,0.140777,0.189320,0.635922,0.648515
1,1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,...,0.004292,0.030043,13026,0.412017,0.064378,0.154506,0.184549,0.184549,1.210300,1.189873
2,2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,...,0.010453,0.017422,8203,0.543554,0.059233,0.135889,0.097561,0.163763,0.334495,0.380952
3,3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,...,0.029915,0.025641,3340,0.551282,0.111111,0.094017,0.055556,0.188034,0.747863,0.650558
4,4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,...,0.037500,0.062500,14020,0.425000,0.200000,0.112500,0.075000,0.187500,0.762500,0.924242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,49995,0.344828,0.169972,0.192308,180,0.509915,200,0.738007,53,...,0.036900,0.007380,8772,0.468635,0.081181,0.099631,0.132841,0.217712,0.195572,0.150142
49996,49996,49996,0.417989,0.389163,0.392060,190,0.935961,187,0.935000,314,...,0.005000,0.015000,1395,0.485000,0.050000,0.170000,0.155000,0.140000,1.570000,1.546798
49997,49997,49997,0.352174,0.239645,0.242152,230,0.680473,227,0.685801,279,...,0.009063,0.012085,8513,0.510574,0.039275,0.108761,0.120846,0.220544,0.842900,0.825444
49998,49998,49998,0.334764,0.331915,0.329810,232,0.987234,230,0.966387,281,...,0.004202,0.008403,4295,0.567227,0.042017,0.117647,0.100840,0.172269,1.180672,1.195745


Let's see how long generating some of the descriptors takes:

In [38]:
#iFeature properties

protein = iFeatureOmegaCLI.iProtein('meso_50k.fasta')
protein.display_feature_types()


        ----- Available feature types ------        
        
        AAC                                                Amino acid composition
        EAAC                                               Enhanced amino acid composition
        CKSAAP type 1                                      Composition of k-spaced amino acid pairs type 1 - normalized
        CKSAAP type 2                                      Composition of k-spaced amino acid pairs type 2 - raw count
        DPC type 1                                         Dipeptide composition type 1 - normalized
        DPC type 2                                         Dipeptide composition type 2 - raw count
        TPC type 1                                         Tripeptide composition type 1 - normalized
        TPC type 2                                         Tripeptide composition type 1 - raw count
        CTDC                                               Composition
        CTDT                                      

In [39]:
"""
this list comes from a combination of reading through the features and determining which might be useful
and timing some of the feature generations. those that took more than 30ish seconds were eliminated
Also removed those that have really high dimensionality (>4000)
"""

feature_list = ['AAC', 'GAAC', 'DistancePair',
               'DPC type 1', 'CTDC', 'CTDT', 'CTDD', 'CTriad',
                'CKSAAGP type 1', 'PseKRAAC type 1', 'APAAC', 'QSOrder']

In [40]:
import time

In [41]:
def time_feature_generation(dataframe, file_name:str, descriptor:str):
    
    """
    Times how long it takes to generate a specific descriptor.
    
    Parameters
    ----------
    Pandas dataframe, fasta file name, descriptor name (string).

    Returns
    -------
    Time to generate descriptor.
    """

    # Record the current time
    start_time = time.time()

    # Code to be timed goes here
    create_new_dataframe(dataframe, file_name, descriptors=[descriptor])

    # Record the time again and calculate the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Print the elapsed time
    return elapsed_time

In [42]:
time_feature_generation(df, 'meso_50k.fasta', 'QSOrder')

File imported successfully.


<ipython-input-34-58a7bf96e801>:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, features, how='outer', left_index=True, right_index=True)


50.5052011013031

In [43]:
def time_feature_dict(dataframe, file_name:str, feature_list):
    
    """
    Iterates through dictionary of protein descriptors and returns the time it takes to run the code.
    
    Parameters
    ----------
    Pandas dataframe, fasta file name, descriptors (list of strings).

    Returns
    -------
    Dictionary of descriptors and time to generate them.
    """
    
    protein = iFeatureOmegaCLI.iProtein(file_name)
    
    time_dict = {}
    
    for feature in feature_list:
        time_dict[feature] = time_feature_generation(dataframe, file_name, feature)
    
    return time_dict

In [44]:
time_feature_dict(df, 'meso_50k.fasta', feature_list)

File imported successfully.


<ipython-input-34-58a7bf96e801>:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, features, how='outer', left_index=True, right_index=True)


File imported successfully.


<ipython-input-34-58a7bf96e801>:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, features, how='outer', left_index=True, right_index=True)


File imported successfully.


...

File imported successfully.


<ipython-input-34-58a7bf96e801>:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, features, how='outer', left_index=True, right_index=True)


File imported successfully.


<ipython-input-34-58a7bf96e801>:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, features, how='outer', left_index=True, right_index=True)


{'AAC': 12.905656099319458,
 'GAAC': 13.434737205505371,
 'DistancePair': 17.440794229507446,
 'DPC type 1': 140.17239379882812,
 'CTDC': 10.232621908187866,
 'CTDT': 84.93229627609253,
 'CTDD': 136.90583610534668,
 'CTriad': 62.94322204589844,
 'CKSAAGP type 1': 66.76271605491638,
 'PseKRAAC type 1': 7.033976793289185,
 'APAAC': 31.77904224395752,
 'QSOrder': 49.19735503196716}

In [46]:
df = create_new_dataframe(df, 'meso_50k.fasta', descriptors=[feature for feature in feature_list])

File imported successfully.


<ipython-input-34-58a7bf96e801>:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, features, how='outer', left_index=True, right_index=True)
<ipython-input-34-58a7bf96e801>:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, features, how='outer', left_index=True, right_index=True)
<ipython-input-34-58a7bf96e801>:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, features, how='outer', left_index=True, right_index=True)
<ipython-input-34-58a7bf96e801>:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.

In [47]:
df

,level_0,index_x,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,...,QSOrder_Grantham.Xr.T,QSOrder_Grantham.Xr.W,QSOrder_Grantham.Xr.Y,QSOrder_Grantham.Xr.V,QSOrder_Schneider.Xd.1,QSOrder_Schneider.Xd.2,QSOrder_Schneider.Xd.3,QSOrder_Grantham.Xd.1,QSOrder_Grantham.Xd.2,QSOrder_Grantham.Xd.3
0,0,0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,...,0.000034,0.000007,0.000021,0.000075,0.301377,0.329472,0.295350,0.323335,0.351859,0.324802
1,1,1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,...,0.000032,0.000003,0.000020,0.000055,0.320640,0.307105,0.307201,0.326660,0.339373,0.333964
2,2,2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,...,0.000039,0.000007,0.000012,0.000080,0.315597,0.328792,0.295681,0.348107,0.342581,0.309310
3,3,3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,...,0.000040,0.000024,0.000020,0.000074,0.310198,0.320598,0.278698,0.333612,0.341294,0.325091
4,4,4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,...,0.000020,0.000031,0.000051,0.000071,0.273665,0.285284,0.235342,0.335457,0.374223,0.290310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,49995,49995,0.344828,0.169972,0.192308,180,0.509915,200,0.738007,...,0.000036,0.000024,0.000005,0.000053,0.313617,0.309291,0.314381,0.322024,0.341910,0.336064
49996,49996,49996,49996,0.417989,0.389163,0.392060,190,0.935961,187,0.935000,...,0.000021,0.000004,0.000011,0.000107,0.301560,0.325260,0.298274,0.334512,0.342559,0.322925
49997,49997,49997,49997,0.352174,0.239645,0.242152,230,0.680473,227,0.685801,...,0.000042,0.000007,0.000009,0.000057,0.310008,0.316547,0.318922,0.331502,0.332637,0.335859
49998,49998,49998,49998,0.334764,0.331915,0.329810,232,0.987234,230,0.966387,...,0.000021,0.000003,0.000006,0.000064,0.307835,0.313564,0.304121,0.345496,0.330889,0.323611


# Scratch work for above function

In [48]:
# sequence = "meso_input_copy.fasta.txt"
# string_to_remove = "<unknown description>"

# with open(sequence, "r") as file:
#     content = file.read()
    
# # Remove the string
# new_content = content.replace(string_to_remove, "")

# with open(sequence, "w") as file:
#     file.write(new_content)


In [49]:
df['t_protein_len'].describe()

count    50000.000000
mean       261.367700
std         69.675548
min         27.000000
25%        226.000000
50%        260.000000
75%        311.000000
max        400.000000
Name: t_protein_len, dtype: float64

The bit-score provides a better rule-of-thumb for inferring homology. For average length proteins, a bit score of 50 is almost always significant. A bit score of 40 is only significant (E() < 0.001) in searches of protein databases with fewer than 7000 entries. Increasing the score by 10 bits increases the significance 210=1000-fold, so 50 bits would be significant in a database with less than 7 million entries (10 times SwissProt, and within a factor of 3 of the largest protein databases). Thus, the NCBI Blast web site uses a color code of blue for alignment with scores between 40–50 bits; and green for scores between 50–80 bits. In the yeast vs human example, the alignments with less than 20% identity had scores ranging from 55 – 170 bits. Except for very long proteins and very large databases, 50 bits of similarity score will always be statistically significant and is a much better rule-of-thumb for inferring homology in protein alignments.

Pearson et al., 2013: An Introduction to Sequence Similarity (“Homology”) Searching

In [50]:
df['protein_match'] = ((df['bit_score'] > 50 ) & (df['query_align_cov'] > 0.8))

In [51]:
df['protein_match'].value_counts()

True     25887
False    24113
Name: protein_match, dtype: int64

In [52]:
df

,level_0,index_x,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,...,QSOrder_Grantham.Xr.W,QSOrder_Grantham.Xr.Y,QSOrder_Grantham.Xr.V,QSOrder_Schneider.Xd.1,QSOrder_Schneider.Xd.2,QSOrder_Schneider.Xd.3,QSOrder_Grantham.Xd.1,QSOrder_Grantham.Xd.2,QSOrder_Grantham.Xd.3,protein_match
0,0,0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,...,0.000007,0.000021,0.000075,0.301377,0.329472,0.295350,0.323335,0.351859,0.324802,False
1,1,1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,...,0.000003,0.000020,0.000055,0.320640,0.307105,0.307201,0.326660,0.339373,0.333964,True
2,2,2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,...,0.000007,0.000012,0.000080,0.315597,0.328792,0.295681,0.348107,0.342581,0.309310,True
3,3,3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,...,0.000024,0.000020,0.000074,0.310198,0.320598,0.278698,0.333612,0.341294,0.325091,False
4,4,4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,...,0.000031,0.000051,0.000071,0.273665,0.285284,0.235342,0.335457,0.374223,0.290310,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,49995,49995,0.344828,0.169972,0.192308,180,0.509915,200,0.738007,...,0.000024,0.000005,0.000053,0.313617,0.309291,0.314381,0.322024,0.341910,0.336064,False
49996,49996,49996,49996,0.417989,0.389163,0.392060,190,0.935961,187,0.935000,...,0.000004,0.000011,0.000107,0.301560,0.325260,0.298274,0.334512,0.342559,0.322925,True
49997,49997,49997,49997,0.352174,0.239645,0.242152,230,0.680473,227,0.685801,...,0.000007,0.000009,0.000057,0.310008,0.316547,0.318922,0.331502,0.332637,0.335859,False
49998,49998,49998,49998,0.334764,0.331915,0.329810,232,0.987234,230,0.966387,...,0.000003,0.000006,0.000064,0.307835,0.313564,0.304121,0.345496,0.330889,0.323611,True


In [53]:
#get rid of stuff that isn't quantitative

df = df.drop(columns = ['Unnamed: 0','thermo_index', 'm_protein_seq', 't_protein_seq',
                        'm_protein_desc', 't_protein_desc'])

In [54]:
df.head()

,level_0,index_x,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,...,QSOrder_Grantham.Xr.W,QSOrder_Grantham.Xr.Y,QSOrder_Grantham.Xr.V,QSOrder_Schneider.Xd.1,QSOrder_Schneider.Xd.2,QSOrder_Schneider.Xd.3,QSOrder_Grantham.Xd.1,QSOrder_Grantham.Xd.2,QSOrder_Grantham.Xd.3,protein_match
0,0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,...,0.000007,0.000021,0.000075,0.301377,0.329472,0.295350,0.323335,0.351859,0.324802,False
1,1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,...,0.000003,0.000020,0.000055,0.320640,0.307105,0.307201,0.326660,0.339373,0.333964,True
2,2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,...,0.000007,0.000012,0.000080,0.315597,0.328792,0.295681,0.348107,0.342581,0.309310,True
3,3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,...,0.000024,0.000020,0.000074,0.310198,0.320598,0.278698,0.333612,0.341294,0.325091,False
4,4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,...,0.000031,0.000051,0.000071,0.273665,0.285284,0.235342,0.335457,0.374223,0.290310,True


From pairplot below, it looks like query_align_cov_16s and subject_align_cov_16s dont correlate with target. Let's remove them.

In [55]:
df = df.drop(columns = ['query_align_cov_16s', 'subject_align_cov_16s'])

In [56]:
# sns.pairplot(df.sample(500),height = 4)

In [57]:
df.describe()

,level_0,index_x,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,...,QSOrder_Grantham.Xr.T,QSOrder_Grantham.Xr.W,QSOrder_Grantham.Xr.Y,QSOrder_Grantham.Xr.V,QSOrder_Schneider.Xd.1,QSOrder_Schneider.Xd.2,QSOrder_Schneider.Xd.3,QSOrder_Grantham.Xd.1,QSOrder_Grantham.Xd.2,QSOrder_Grantham.Xd.3
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,24999.500000,24999.500000,0.349026,0.276478,0.270792,206.011660,0.791507,205.414920,0.770875,230.63206,...,0.000041,0.000007,0.000014,0.000063,0.308377,0.316125,0.304709,0.334357,0.338000,0.327640
std,14433.901067,14433.901067,0.096608,0.114404,0.116039,64.260901,0.144893,64.430548,0.179955,186.44859,...,0.000013,0.000006,0.000008,0.000018,0.013992,0.014411,0.015390,0.011518,0.012159,0.012242
min,0.000000,0.000000,0.164103,0.081301,0.045977,19.000000,0.284553,19.000000,0.082019,35.00000,...,0.000000,0.000000,0.000000,0.000000,0.129203,0.086857,0.108998,0.276372,0.202020,0.228923
25%,12499.750000,12499.750000,0.293750,0.204545,0.202020,172.000000,0.673288,172.000000,0.656854,118.00000,...,0.000032,0.000003,0.000008,0.000051,0.300626,0.309116,0.297099,0.327045,0.330088,0.319966
50%,24999.500000,24999.500000,0.325000,0.250000,0.244648,210.000000,0.811321,209.000000,0.806584,186.00000,...,0.000040,0.000006,0.000012,0.000062,0.309352,0.317607,0.306206,0.334359,0.338027,0.327910
75%,37499.250000,37499.250000,0.367257,0.308300,0.301639,240.000000,0.923077,240.000000,0.923077,267.00000,...,0.000049,0.000010,0.000018,0.000075,0.317544,0.325292,0.314490,0.341567,0.345976,0.335679
max,49999.000000,49999.000000,0.985075,0.985075,0.985075,400.000000,1.000000,399.000000,1.000000,1788.00000,...,0.000209,0.000076,0.000110,0.000276,0.366035,0.362622,0.356207,0.454076,0.410000,0.391228


Split data into dev and test, and then split that into train and validation.

In [58]:
#drop columns that don't exihibit signficant pearson correlation with bit_score

df = df.drop(columns = ['meso_index', 'meso_protein_int_index', 'local_gap_compressed_percent_id_16s', 
                        'scaled_local_query_percent_id_16s', 'scaled_local_symmetric_percent_id_16s',
                       'bit_score_16s', 'm_ogt', 't_ogt', 'taxa_pair_index', 'thermo_protein_int_index'
                       , 'prot_pair_index', 'ogt_difference'])

In [59]:
#choosing 80/20 split instead of 85/15 because of volume of data

dev, test = sklearn.model_selection.train_test_split(df, test_size=0.15, random_state=1)

train, val = sklearn.model_selection.train_test_split(dev, test_size=0.15, random_state=1)

print(dev.shape)
print(test.shape)
print(train.shape)
print(val.shape)

(42500, 1289)
(7500, 1289)
(36125, 1289)
(6375, 1289)


In [60]:
#ID target and features, separate into separate arrays

target = 'protein_match'
input_features = [columns for columns in df]
input_features.remove(target)

In [61]:
print(input_features)
print(target)

['level_0', 'index_x', 'local_gap_compressed_percent_id', 'scaled_local_query_percent_id', 'scaled_local_symmetric_percent_id', 'query_align_len', 'query_align_cov', 'subject_align_len', 'subject_align_cov', 'bit_score', 'm_protein_len', 't_protein_len', 'index_y', 'AAC_A_x', 'AAC_C_x', 'AAC_D_x', 'AAC_E_x', 'AAC_F_x', 'AAC_G_x', 'AAC_H_x', 'AAC_I_x', 'AAC_K_x', 'AAC_L_x', 'AAC_M_x', 'AAC_N_x', 'AAC_P_x', 'AAC_Q_x', 'AAC_R_x', 'AAC_S_x', 'AAC_T_x', 'AAC_V_x', 'AAC_W_x', 'AAC_Y_x', 'index_x', 'GAAC_alphatic_x', 'GAAC_aromatic_x', 'GAAC_postivecharge_x', 'GAAC_negativecharge_x', 'GAAC_uncharge_x', 'index_y', 'AAC_A_y', 'AAC_C_y', 'AAC_D_y', 'AAC_E_y', 'AAC_F_y', 'AAC_G_y', 'AAC_H_y', 'AAC_I_y', 'AAC_K_y', 'AAC_L_y', 'AAC_M_y', 'AAC_N_y', 'AAC_P_y', 'AAC_Q_y', 'AAC_R_y', 'AAC_S_y', 'AAC_T_y', 'AAC_V_y', 'AAC_W_y', 'AAC_Y_y', 'index_x', 'GAAC_alphatic_y', 'GAAC_aromatic_y', 'GAAC_postivecharge_y', 'GAAC_negativecharge_y', 'GAAC_uncharge_y', 'index_y', 'DP_A', 'DP_C', 'DP_D', 'DP_E', 'DP_F'

In [62]:
#split X and y

dev_X = dev[input_features].values
test_X = test[input_features].values

dev_y = dev[target].values.reshape(-1,1)
test_y = test[target].values.reshape(-1,1)  

print(dev_X.shape, test_X.shape, dev_y.shape, test_y.shape)

(42500, 1372) (7500, 1372) (42500, 1) (7500, 1)


In [63]:
#same thing for training and validation data

train_X = train[input_features].values
val_X = val[input_features].values

train_y = train[target].values.reshape(-1,1)
val_y = val[target].values.reshape(-1,1) 

Scale the data

In [64]:
scaler = sklearn.preprocessing.StandardScaler()
dev_X = scaler.fit_transform(dev_X)
test_X = scaler.fit_transform(test_X)
train_X = scaler.fit_transform(train_X)
val_X = scaler.fit_transform(val_X)

Train the model

In [65]:
#Random Forest

model = sklearn.ensemble.RandomForestClassifier(n_estimators=150, max_depth=None, max_samples=0.5,
                                                max_features=0.5, min_weight_fraction_leaf=0.000215,
                                               min_samples_split=10)
# model = sklearn.ensemble.RandomForestClassifier()
model.fit(train_X, train_y.ravel())

RandomForestClassifier(max_features=0.5, max_samples=0.5, min_samples_split=10,
                       min_weight_fraction_leaf=0.000215, n_estimators=150)

Test the model, report relevant statistics

In [66]:
score = model.score(val_X, val_y)
print('Model score is: {}'.format(score))

preds = model.predict(test_X)
print(preds)

Model score is: 0.9927843137254901
[False False False ... False  True False]


In [67]:
#confusion matrix

confusion_matrix = sklearn.metrics.confusion_matrix(preds, test_y)
sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix).plot()

Convert the above code into functions.

In [37]:
def train_model(dataframe, columns = [],  target = []):
    """
    Takes dataframe and splits it into a training and testing set. 
    Note: Data is called train and test, but this test set is currently
    closer to a validation set. Keeping nomenclature to keep model robust.
    Trains a KNN classifier model with selected data.
    
    Params
    ----------
    dataframe: Pandas dataframe 
    columns: list of strings, representing input features
    target: list of strings, representing target feature(s)

    Returns
    -------
    -Sk-learn model object
    -train data (features)
    -train data (target)
    -validation data (features)
    -validation data (target)
    """
    #split data
    dev, test = sklearn.model_selection.train_test_split(dataframe, test_size=0.15, random_state=1)
    
    #test input arguments
    assert "pandas.core.frame.DataFrame" in str(type(dev))
    assert "pandas.core.frame.DataFrame" in str(type(test))
    assert "str" in str(type(columns[0]))
    assert "str" in str(type(target[0]))
   
    #split into input and output feature(s)
    dev_X = dev[columns].values
    test_X = test[columns].values

    dev_y = dev[target].values.reshape(-1,1)
    test_y = test[target].values.reshape(-1,1)
    
    #scale data
    scaler = sklearn.preprocessing.StandardScaler()
    dev_X = scaler.fit_transform(dev_X)
    test_X = scaler.fit_transform(test_X)
    
    #train model
    model = sklearn.ensemble.RandomForestClassifier()
    model = model.fit(dev_X, dev_y.ravel())
    
    return model, dev_X, dev_y, test_X, test_y
    

In [38]:
model, dev_X, dev_y, test_X, test_y = train_model(df, columns = input_features, 
                                          target='protein_match')

In [39]:
class TestModelTraining(unittest.TestCase):
    
    def test_invalid_inputs(self):
    
        #test that input data type is correct
        
        try:
            train_model([1,2,3], columns = 'string', target = 'string')
            self.assertTrue(False)
        except AssertionError:
            self.assertTrue(True)
    
#     def test_input_distro(self):
        
#         #test that dev and test features have similar Jensen Shannon Distribution
        
#         JSD = (
#             scipy.stats.bootstrap((train_reg(split_data(df)[0], split_data(df)[1],
#                                 columns = input_features, target=target)[1], train_reg(split_data(df)[0], 
#                                 split_data(df)[1], columns = input_features, target=target)[3]), 
#                                   JSD_dev_and_test, n_resamples=1000, 
#                                   batch=5, method='percentile')
#         )

#         div = JSD.confidence_interval[1]
        
#         #asserts that the divergence between data sets is sufficiently low
#         assert abs(div) < 0.3, "Warning! High JSD between dev and test set!"
        
    def test_output_format(self):
        
        #asserts that function returns 4 objects to be assigned to pearson_corr, model, test_X, test_y
        assert len(train_model(df, columns = input_features, 
                                          target = target)) == 5
        

suite = unittest.TestLoader().loadTestsFromTestCase(TestModelTraining)
_ = unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 12.454s

OK


In [40]:
#need to fix this

def evaluate_model(model, test_X, test_y):
    
    """
    Takes a trained model and test data and tests the model.
    
    Params
    ----------
    model: sklearn.neighbors.KNeighborsClassifier
    test_X: numpy array
    test_y: numpy array

    Returns
    -------
    Vector of predictions based on the model (numpy array)
    """
    
    #test input arguments
    assert "sklearn" in str(type(model))
    assert "numpy.ndarray" in str(type(test_X))
    assert "numpy.ndarray" in str(type(test_y))
    
    preds = model.predict(test_X)
    
    return preds

In [41]:
preds = evaluate_model(model, test_X, test_y)
preds

array([False, False, False, ..., False,  True, False])

In [42]:
class TestModelPerformance(unittest.TestCase):

    def test_asserts(self):
        model, _, _, test_X, test_y = train_model(
            df, columns=input_features, target='protein_match'
        )
        # assert that input types are correct
        with self.assertRaises(AssertionError):
            evaluate_model(model, [1, 2, 3], test_y)
            
    def test_model_output(self):
        model, _, _, test_X, test_y = train_model(
            df, columns=input_features, target='protein_match'
        )
        # assert output type is correct
        output = evaluate_model(model, test_X, test_y)
        self.assertIsInstance(output, np.ndarray)
        
    def test_pred_dimension(self):
        model, _, _, test_X, test_y = train_model(df, 
            columns=input_features, target='protein_match'
        )
        # want to check that the number of predictions is equal to the number of test examples
        preds = evaluate_model(model, test_X, test_y)
        self.assertEqual(len(test_y), len(preds))

suite = unittest.TestLoader().loadTestsFromTestCase(TestModelPerformance)
_ = unittest.TextTestRunner().run(suite)


...
----------------------------------------------------------------------
Ran 3 tests in 34.427s

OK


A good test here can be adding outliers and seeing how much the score and predictions change (probably for the above function).

In [43]:
def plot_model(model, test_X, test_y):
    """
    Takes a test KNN Classifier model and plots the confusion matrix.
    
    Params
    ----------
    model: sklearn.neighbors.KNeighborsClassifier
    test_X: numpy array
    test_y: numpy array

    Returns
    -------
    -Confusion predictions vs. observations
    -Model score
    """
    
    #test input arguments
    assert "sklearn" in str(type(model))
    assert "numpy.ndarray" in str(type(test_X))
    assert "numpy.ndarray" in str(type(test_y))
    
    score = model.score(test_X, test_y)
    preds = evaluate_model(model, test_X, test_y)
   
    # plot confusion matrix
    confusion_matrix = sklearn.metrics.confusion_matrix(preds, test_y)
    cm_plot = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix)
    
    cm_plot.plot(cmap=plt.cm.Blues)
    cm_plot.ax_.set_title('Confusion Matrix')
    
    return score
    

In [44]:
#make some appropriate display labels here

plot_model(model, test_X, test_y)

0.9952

In [45]:
#wrapper function

def RF_wrapper(dataframe):
    """
    Takes dataframe and runs it through kNN model.
    
    Params
    ----------
    dataframe: Pandas dataframe 

    Returns
    -------
    -Target feature predictions
    -Parity plot
    """
    
    assert 'pandas.core.frame.DataFrame' in str(type(dataframe))

    #user inputs target feature
    target = 'protein_match'
    
    #define input features
    input_features = [columns for columns in dataframe]
    
    input_features.remove(target)

    #train the model based off data split
    model, dev_X, dev_y, test_X, test_y = train_model(
                                        dataframe, columns=input_features, 
                                        target=target
                                                   )
    
    #test the model and return predictions
    preds = evaluate_model(model, test_X, test_y)

    #plot the results of the model
    plot_model(model, test_X, test_y)
    
    return preds

In [46]:
RF_wrapper(df)

array([False, False, False, ..., False,  True, False])

In [47]:
class TestWrapper(unittest.TestCase):
    
    def test_wrapper_input(self):
        #test that input data type is correct
        try:
            RF_wrapper([1,2,3])
            self.assertTrue(False)
        except AssertionError:
            self.assertTrue(True)

    def test_wrapper_output(self):
        model, _, _, test_X, test_y = train_model(
            df, 
            columns=input_features, target='protein_match'
        )
        # assert output type is correct
        output = evaluate_model(model, test_X, test_y)
        self.assertIsInstance(output, np.ndarray)
        
    def test_output_dimension(self):
        model, _, _, test_X, test_y = train_model(
            df, 
            columns=input_features, target='protein_match'
        )
        # want to check that the number of predictions is equal to the number of test examples
        preds = evaluate_model(model, test_X, test_y)
        self.assertEqual(len(test_y), len(preds))
        

suite = unittest.TestLoader().loadTestsFromTestCase(TestWrapper)
_ = unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 24.401s

OK


**IGNORE BELOW**

In [48]:
# #plot empirical distribution of scaled_local_query_percent_id

# target = df['protein_match']
# #create histplot
# fig, ax = plt.subplots()

# sns.histplot(data = df, x = target)

# ax.set_title('Sample Histogram', fontsize=16)
# ax.set_xlabel('Class', fontsize=14)
# ax.set_ylabel('Count', fontsize=14)

Things I've tried to improve model: 

1. drop bit scores over 1000
2. Switch to 85/15 train/test split
3. Lasso regression - not great
4. Ridge regression - R2 = 0.87, about the same as normal Linear regression
5. KNN regression (n_neighbors optimized at 8) gives best result, R2 = ~0.93 --> 
*got it up to .942 by removing a few features
6. Decision tree regressor was slightly worse than KNN
7. Input DT classifier and RF classifier. RF classifier has best performance
8. Changed target to binary protein function match